Index: 01

Date: 2021/11/07

In [1]:
# !pip install geopandas

In [64]:
import pandas as pd
import numpy as np
import geopandas
import shapely
from geopandas.tools import sjoin
from os import listdir

Notes: If having trouble installing geopandas (/fiona/GDAL), please visit https://www.lfd.uci.edu/~gohlke/pythonlibs/#gdal and https://www.lfd.uci.edu/~gohlke/pythonlibs/#fiona to find supported whls (Note: cp37 means python version 3.7). Then `pip install GDAL-xxx.whl`, `set GDAL_VERSION=xxx`, `pip install Fiona-xxx.whl` , and lastly `pip install geopandas`.

If there is problem with shapely.geos, first pip uninstall shapely within conda prompt, then conda install shapely.

In [59]:
# Import farm coordinates from the excel file as a dataframe.
farm_coordinates = pd.read_excel('group_a_addresses_geocoded_manual_updates.xlsx', index_col=0)

# Change the points dataframe to a GeoDataFrame with geometry given by longitude and latitude variables.
point_all = geopandas.GeoDataFrame(
    farm_coordinates, 
    geometry=geopandas.points_from_xy(farm_coordinates.longitude, farm_coordinates.latitude))

# Set Coordinate Reference System (CRS) to be epsg:4326, or the WGS84 latitude-longitude projection.
point_all = point_all.set_crs(epsg=4326)

In [60]:
def merge_point_poly(poly_file_path, point_all):
    """
    Find polygons that have at least a target point inside their boundaries.
    Input: poly_file_path: str, file path of the shapefile of polygons.
           point_all: GeoDataFrame, GeoDataFrame of points.
    Output: GeoDataFrame, GeoDataFrame of polygons that has at least one target point inside.
    """
    # Construct a GeoDataFrame of polygons from the shapefile.
    poly = geopandas.GeoDataFrame.from_file(poly_file_path)
    # Transform all geometries in a point_all to crs of the polygon shapfile. 
    point = point_all.to_crs(poly.crs)
    # Spatial join of point and poly, and keep geometry of poly as the geometry of the new shapefile.
    # (It only keeps the geometry from the left GeoDataFrame. 
    pointPolys = sjoin(poly, point, how='inner')
    # Select those entries without nan.
    # pointPolys[~(pointPolys.index_left.isna())]
    return pointPolys

In [61]:
a = merge_point_poly('il/il001/clu_public_a_il001.shp', point_all)

In [69]:
# Loop through all shapefiles in Illinois.
list_file = listdir('il')
for i in range(len(list_file)):
    file_name = list_file[i]
    if i ==0:
        output = merge_point_poly('il/'+file_name+'/clu_public_a_'+file_name+'.shp', point_all)
    else:
        m = merge_point_poly('il/'+file_name+'/clu_public_a_'+file_name+'.shp', point_all)
        # Appended geometry columns needs to have the same CRS.
        m.to_crs(output.crs, inplace=True)
        output = output.append(m)

In [72]:
output.reset_index(drop=True, inplace=True)

In [74]:
output.to_file("fields.shp")

<ipython-input-74-d429184222b5>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  output.to_file("fields.shp")
